In [134]:
import sys
sys.path.append('..')

import math
import numpy as np

import metrics

In [8]:
def check_f(vals, fref, fme):
    aref = np.empty((len(vals),))
    ame = np.empty((len(vals),))
    for i in range(len(vals)):
        aref[i] = fref(vals[i])
        ame[i] = fme(vals[i])
    
    return (aref, ame, metrics.tdist(aref, ame))

$$
x! = 
\begin{cases}
    1 & \text{if } x = 0\\
    x * (x - 1)! & \text{otherwise}
\end{cases}
$$

In [9]:
#Factorial

NFACTS = 50
facts = [1] * NFACTS
for i in range(2, NFACTS):
    facts[i] = i * facts[i - 1]

print(facts[:10])

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]


$$
x^n = 
\begin{cases}
    1 & \text{if } n = 0\\
    (x^2)^{n/2} & \text{if } x \% 2 = 0\\
    x * (x^2)^{(n-1)/2} & \text{if } x \% 2 = 1
\end{cases}
$$

In [13]:
# Power

def fast_pow(x, n):
    if n == 0:
        return 1
    elif n == 1:
        return x
    elif n % 2 == 0:
        return fast_pow(x*x, n//2)
    elif n % 2 == 1:
        return x * fast_pow(x*x, n//2)

$$e = \sum_{n=0}^{\infty} \frac{1}{n!}$$

In [14]:
#Euler

def approx_euler(n):
    res = 0
    for i in range(n):
        res += 1 / facts[i]
    return res

EULER = approx_euler(20)
print(EULER)
print(np.exp(1))
print(metrics.tdist(EULER, np.exp(1)))

2.7182818284590455
2.718281828459045
4.440892098500626e-16


Taylor approximation of $e^x$ close to $0$:

$$e^x = \sum_{n=0}^{\infty} \frac{x^n}{n!} = 1 + x + \frac{x^2}{2} + \frac{x^3}{6} + \text{...}$$
$$e^{a+b} = e^a * e^b$$
$$e^{-x} = \frac{1}{e^x}$$

In [18]:
## Exponential

exp_tables = []
for i in range(10):
    v = fast_pow(2, i)
    exp_tables.append(fast_pow(EULER, v))

def my_exp(x):
    if x < 0:
        return 1. / my_exp(-x)
    
    if x > 1:
        xi = int(x)
        xf = x - xi
        res = my_exp(xf)
        p = 0
        
        while xi:
            if xi % 2:
                res *= exp_tables[p]
            p += 1
            xi = xi // 2
        return res
    
    res = 1
    for i in range(1, 20):
        res += fast_pow(x, i) / facts[i]
    return res


ref, me, dist = check_f(np.linspace(-1, 1, 10), np.exp, my_exp)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-25, 25, 2000), np.exp, my_exp)
print(dist)

[0.36787944 0.45942582 0.57375342 0.71653131 0.89483932 1.11751907
 1.39561243 1.742909   2.17662993 2.71828183]
[0.36787944 0.45942582 0.57375342 0.71653131 0.89483932 1.11751907
 1.39561243 1.742909   2.17662993 2.71828183]
7.967349055175334e-16
0.0008970465527435638


Taylor approximation of $ln(1-x)$ close to $0$:

$$ln(1-x) = - \sum_{n=1}^{\infty} \frac{x^n}{n} = - x - \frac{x^2}{2} - \frac{x^3}{3} - \text{...}$$
$$ln(ab) = ln(a) + ln(b)$$
$$ln(a^b) = bln(a)$$
$$ln(y) = n * ln(2) + ln(x), \space y = 2^nx$$

In [99]:
def get_msb(x):
    p = 0
    n = 1
    while n <= x:
        p += 1
        n *= 2
    return p - 1


def my_ln_02(x):    
    x = 1 - x
    res = 0
    for i in range(1, 20):
        res -= fast_pow(x, i) / i
    return res

#2 = 1.001^6931.81837341456 
LN2 = 6931.81837341456 * my_ln_12(1.0001)


def my_ln(x):
    if x <= 2:
        return my_ln_02(x)
    
    # msb (most significant byte) is the biggest power of 2 <= x
    n = get_msb(int(x))
    x /= fast_pow(2, n)
    return n * LN2 + my_ln_12(x)
    
    
ref, me, dist = check_f(np.linspace(0.1, 1, 10), np.log, my_ln)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(1, 1000, 2000), np.log, my_ln)
print(dist)

[-2.30258509 -1.60943791 -1.2039728  -0.91629073 -0.69314718 -0.51082562
 -0.35667494 -0.22314355 -0.10536052  0.        ]
[-2.2572709  -1.60696941 -1.20385216 -0.91628645 -0.69314709 -0.51082562
 -0.35667494 -0.22314355 -0.10536052  0.        ]
0.04538153919839533
0.1498549109482484


$$x^y = \exp(\log(x^y)) = \exp(y * \log(x))$$
$$x^{-y} = \frac{1}{x^y}$$

In [158]:
import functools

def my_pow(x, y):
    if y < 0:
        return 1 / my_pow(x, -y)
        
    return my_exp(y * my_ln(x))


ref, me, dist =  check_f(np.linspace(-1, 1, 10), 
                         functools.partial(np.power, 3.),
                         functools.partial(my_pow, 3.))

print(ref)
print(me)
print(dist)

ref, me, dist =  check_f(np.linspace(-10, 10, 2000), 
                         functools.partial(np.power, 4.7),
                         functools.partial(my_pow, 4.7))
print(dist)

[0.33333333 0.425506   0.54316607 0.69336127 0.88508815 1.12983096
 1.44224957 1.84105755 2.35014311 3.        ]
[0.33333332 0.42550599 0.54316606 0.69336127 0.88508815 1.12983097
 1.44224959 1.84105758 2.35014317 3.0000001 ]
1.2093071443462362e-07
1.2911417773478193e-07


Nilakantha series:

$$\pi = 3 + \sum_{n=0}^\infty \frac{(-1)^n * 4}{(2n+2) * (2n+3) * (2n+4)}$$
$$\pi = 3 + \frac{4}{2 * 3 * 4} - \frac{4}{4 * 5 * 6} + \frac{4}{6 * 7 * 8} - \frac{4}{8 * 9 * 10} + \text{...}$$

In [157]:
def get_pi(n):
    
    res = 3.
    
    for i in range(n):
        num = 4. if i % 2 == 0 else -4.
        d1 = (i + 1) * 2.        
        res += num / (d1 * (d1 + 1) * (d1 + 2))
        
    return res
    
PI = get_pi(50000)
print(PI)
print(math.pi)
print((PI - math.pi) ** 2)

3.141592653589785
3.141592653589793
6.389773332290196e-29


$$\sin(-x) = - \sin(x)$$
$$\sin(x + 2k\pi) = \sin(x), \space \forall k \in \mathbb{N}$$

Taylor approximation of $\sin$ close to $0$:

$$\sin(x) = \sum_{n=0}^{\infty} \frac{(-1)^n}{(2n + 1)!} x^{2n+1} = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \text{...}$$

In [166]:
PIT2 = PI * 2

def my_sin(x):
    if x < 0:
        return - my_sin(-x)
    
    x = x - int(x / PIT2) * PIT2
    
    if x > PI:
        x = x - PIT2  
    
    res = x
    p = x
    x2 = x*x
    for n in range(1, 15):
        p *= x2
        num = p if n % 2 == 0 else -p
        res += num / facts[2 * n + 1]
    return res
    

ref, me, dist = check_f(np.linspace(0, 2 * math.pi, 10), np.sin, my_sin)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-100, 100, 2000), np.sin, my_sin)
print(dist)

[ 0.00000000e+00  6.42787610e-01  9.84807753e-01  8.66025404e-01
  3.42020143e-01 -3.42020143e-01 -8.66025404e-01 -9.84807753e-01
 -6.42787610e-01 -2.44929360e-16]
[ 0.00000000e+00  6.42787610e-01  9.84807753e-01  8.66025404e-01
  3.42020143e-01 -3.42020143e-01 -8.66025404e-01 -9.84807753e-01
 -6.42787610e-01  1.59872116e-14]
2.7020985982140625e-14
4.704554106082915e-12


$$\cos(-x) = \cos(x)$$
$$\cos(x + 2k\pi) = \cos(x), \space \forall k \in \mathbb{N}$$

Taylor approximation of $\cos$ close to $0$:

$$\cos(x) = \sum_{n=0}^{\infty} \frac{(-1)^n}{(2n)!} x^{2n} = 1 - \frac{x^2}{2!} + \frac{x^4}{4!} - \frac{x^6}{6!} + \text{...}$$

In [167]:
def my_cos(x):
    if x < 0:
        x = -x
    
    x = x - int(x / PIT2) * PIT2
    
    if x > PI:
        x = x - PIT2  
    
    res = 1
    p = 1
    x2 = x*x
    for n in range(1, 15):
        p *= x2
        num = p if n % 2 == 0 else -p
        res += num / facts[2 * n]
    return res
    

ref, me, dist = check_f(np.linspace(0, 2 * math.pi, 10), np.cos, my_cos)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-100, 100, 2000), np.cos, my_cos)
print(dist)

[ 1.          0.76604444  0.17364818 -0.5        -0.93969262 -0.93969262
 -0.5         0.17364818  0.76604444  1.        ]
[ 1.          0.76604444  0.17364818 -0.5        -0.93969262 -0.93969262
 -0.5         0.17364818  0.76604444  1.        ]
2.4354991743539337e-14
4.760877033257799e-12


$$\tan(x) = \frac{\sin(x)}{\cos(x)}$$

In [174]:
def my_tan(x):
    return my_sin(x) / my_cos(x)
    
ref, me, dist = check_f(np.linspace(0, 2 * math.pi, 10), np.tan, my_tan)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-100, 100, 2000), np.tan, my_tan)
print(dist)

[ 0.00000000e+00  8.39099631e-01  5.67128182e+00 -1.73205081e+00
 -3.63970234e-01  3.63970234e-01  1.73205081e+00 -5.67128182e+00
 -8.39099631e-01 -2.44929360e-16]
[ 0.00000000e+00  8.39099631e-01  5.67128182e+00 -1.73205081e+00
 -3.63970234e-01  3.63970234e-01  1.73205081e+00 -5.67128182e+00
 -8.39099631e-01  1.59872116e-14]
5.451569792762914e-13
8.896578884151276e-06


$$\sinh(x) = \frac{e^x - e^{-x}}{2}$$
$$e^{-x} = \frac{1}{e^x}$$

In [173]:
def my_sinh(x):
    ex = my_exp(x)
    return (ex - 1/ex) / 2

ref, me, dist = check_f(np.linspace(-1, 1, 10), np.sinh, my_sinh)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-20, 20, 2000), np.sinh, my_sinh)
print(dist)

[-1.17520119 -0.85860205 -0.58457779 -0.33954056 -0.11133988  0.11133988
  0.33954056  0.58457779  0.85860205  1.17520119]
[-1.17520119 -0.85860205 -0.58457779 -0.33954056 -0.11133988  0.11133988
  0.33954056  0.58457779  0.85860205  1.17520119]
7.721838581466776e-16
3.835947746920957e-06


$$\cosh(x) = \frac{e^x + e^{-x}}{2}$$
$$e^{-x} = \frac{1}{e^x}$$

In [175]:
def my_cosh(x):
    ex = my_exp(x)
    return (ex + 1/ex) / 2

ref, me, dist = check_f(np.linspace(-1, 1, 10), np.cosh, my_cosh)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-20, 20, 2000), np.cosh, my_cosh)
print(dist)

[1.54308063 1.31802788 1.15833121 1.05607187 1.00617919 1.00617919
 1.05607187 1.15833121 1.31802788 1.54308063]
[1.54308063 1.31802788 1.15833121 1.05607187 1.00617919 1.00617919
 1.05607187 1.15833121 1.31802788 1.54308063]
4.965068306494546e-16
3.826703249726514e-06


$$\tanh(x) = \frac{\sinh(x)}{\cosh(x)} = \frac{e^{2x} - 1}{e^{2x} + 1}$$

In [194]:
def my_tanh(x):
    ex = my_exp(x)
    e2x = ex * ex
    return (e2x - 1) / (e2x + 1)

ref, me, dist = check_f(np.linspace(-1, 1, 10), np.tanh, my_tanh)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(-100, 100, 2000), np.tanh, my_tanh)
print(dist)

[-0.76159416 -0.65142936 -0.5046724  -0.32151274 -0.11065611  0.11065611
  0.32151274  0.5046724   0.65142936  0.76159416]
[-0.76159416 -0.65142936 -0.5046724  -0.32151274 -0.11065611  0.11065611
  0.32151274  0.5046724   0.65142936  0.76159416]
7.141332153553285e-16
1.3185541737247534e-15


Newton's method is a technique to find the roots of a real-valued function $f(x)$.  
We start with an initial guess $x_0$
$$x_{n+1} = x_n + \frac{f(x_n)}{f'(x_n)}$$

$f(x_n) = 0$ as $n \to \infty$  

Compute the square root of $a$:  
$$x_{n+1} = x_n - \frac{x_n^2 - a}{2x_n}$$

In [193]:
def my_sqrt(x):
    res = x
    while (x - res*res)**2 > 1e-25:    
        res = res - (res*res - x) / (2 * res)
    return res
    
ref, me, dist = check_f(np.linspace(0, 1, 10), np.sqrt, my_sqrt)
print(ref)
print(me)
print(dist)
ref, me, dist = check_f(np.linspace(0, 100, 2000), np.sqrt, my_sqrt)
print(dist)

[0.         0.33333333 0.47140452 0.57735027 0.66666667 0.74535599
 0.81649658 0.8819171  0.94280904 1.        ]
[0.         0.33333333 0.47140452 0.57735027 0.66666667 0.74535599
 0.81649658 0.8819171  0.94280904 1.        ]
6.308863706625178e-14
2.681804845346326e-13
